# Noisy QCNN Kmnist

In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/home/tak/Github/QEmbedding/')
import torch
from torch import nn
import pennylane as qml
import embedding
import data
from qiskit.providers.fake_provider import FakeGuadalupe
from qiskit_aer.noise import NoiseModel


noisy = FakeGuadalupe()
noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev_fake = qml.device(
    'qiskit.aer',
    wires=8,
    shots=1028,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

/tmp/ipykernel_1508988/110338151.py:14: DeprecationWarning: All fake backend instances based on real device snapshots (`FakeVigo`,`FakeSherbrooke`,...) have been migrated to the `qiskit_ibm_runtime` package. These classes are deprecated as of qiskit 0.46.0 and will be removed in qiskit 1.0.0. To migrate your code, run `pip install qiskit-ibm-runtime` and use `from qiskit_ibm_runtime.fake_provider import FakeExample` instead of `from qiskit.providers.fake_provider import FakeExample`. If you are using a custom fake backend implementation, you don't need to take any action.
  noisy = FakeGuadalupe()


# QCNN Training

In [2]:
batch_size = 15
steps = 200
ansatz = "TTN"
learning_rate = 0.05

In [3]:

class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(8, 24),
            nn.ReLU(),
            nn.Linear(24, 24),
            nn.ReLU(),
            nn.Linear(24, 15)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


model2 = x_transform2()

In [4]:
def statepreparation(x, data):
    if data == 'mnist':
        model2.load_state_dict(torch.load('NQE200_mnist.pt'))
    elif data == 'fashion':
        model2.load_state_dict(torch.load('NQE200_fashion.pt'))
    x = model2(torch.tensor(x))
    embedding.QuantumEmbedding2(x)

def data_reuploading_ansatz(params): #15 params
    for i in range(8):
        qml.RY(params[i], wires=i)
    for i in range(7):
        qml.IsingYY(params[i+8], wires=[i,i+1])

def data_reuploading_embedding(params, x):
    data_reuploading_ansatz(params[0:15])
    embedding.QuantumEmbedding1(x)

@qml.qnode(dev_fake)
def data_reuploading_QCNN_classifier(params, x, data):
    data_reuploading_embedding(params, x)
    embedding.Noisy_QCNN_eight(params[15: 15 + 8], ansatz)
    return qml.expval(qml.PauliZ(4))

@qml.qnode(dev_fake)
def QCNN_classifier(params, x, data):
    statepreparation(x, data)
    embedding.Noisy_QCNN_eight(params, ansatz)
    return qml.expval(qml.PauliZ(4))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost_NQE(weights, X_batch, Y_batch, data):
    preds = [QCNN_classifier(weights, x, data) for x in X_batch]
    return Linear_Loss(Y_batch, preds)

def cost_TQE(weights, X_batch, Y_batch, data):
    preds = [data_reuploading_QCNN_classifier(weights, x, data) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, data):

    num_weights_TQE = 15 + 8
    num_weights_NQE = 8
    
    init_weights = np.random.random(num_weights_TQE, requires_grad = True)
    weights_TQE = init_weights.reshape(23,1)
    weights_NQE = init_weights[:num_weights_NQE].reshape(8,1)
    opt_TQE = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    opt_NQE = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history_TQE, loss_history_NQE = [], []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights_NQE, cost_new_NQE = opt_NQE.step_and_cost(lambda v: cost_NQE(v, X_batch, Y_batch, data),
                                                     weights_NQE)
        weights_TQE, cost_new_TQE = opt_TQE.step_and_cost(lambda v: cost_TQE(v, X_batch, Y_batch, data),
                                                     weights_TQE)
        loss_history_NQE.append(cost_new_NQE)
        loss_history_TQE.append(cost_new_TQE)
        if it % 5 == 0:
            print("--------------------------------------")
            print("iteration: ", it, " cost_NQE: ", cost_new_NQE)
            print("iteration: ", it, " cost_TQE: ", cost_new_TQE)
            
    return loss_history_NQE, loss_history_TQE, weights_NQE, weights_TQE

In [5]:
feature_reduction = 'PCA8'
classes = [0,1]
data_type = 'mnist'
X_train, X_test, Y_train, Y_test = data.data_load_and_process(data_type, feature_reduction=feature_reduction, classes=classes)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [6]:


loss_history_NQE, loss_history_TQE, weights_NQE, weights_TQE = circuit_training(X_train, Y_train, data_type)
f = open('weights and losses mnist.txt', 'a')
f.write(f'Loss History NQE:')
f.write('\n')
f.write(str(loss_history_NQE))
f.write('\n')
f.write(f'Loss History TQE:')
f.write('\n')
f.write(str(loss_history_TQE))
f.write('\n')
f.write(f'Weights NQE:')
f.write('\n')
f.write(str(weights_NQE))
f.write('\n')
f.write(f'Weights TQE:')
f.write('\n')
f.write(str(weights_TQE))
f.write('\n')
f.close()

--------------------------------------
iteration:  0  cost_NQE:  0.5223086900129701
iteration:  0  cost_TQE:  0.5036316472114137
--------------------------------------
iteration:  5  cost_NQE:  0.48910505836575885
iteration:  5  cost_TQE:  0.5011024643320362
--------------------------------------
iteration:  10  cost_NQE:  0.4542153047989624
iteration:  10  cost_TQE:  0.49870298313878075
--------------------------------------
iteration:  15  cost_NQE:  0.4171854734111543
iteration:  15  cost_TQE:  0.49273670557717253
--------------------------------------
iteration:  20  cost_NQE:  0.3767185473411153
iteration:  20  cost_TQE:  0.4902075226977951
--------------------------------------
iteration:  25  cost_NQE:  0.4403372243839169
iteration:  25  cost_TQE:  0.47600518806744496
--------------------------------------
iteration:  30  cost_NQE:  0.4329442282749676
iteration:  30  cost_TQE:  0.5292477302204928
--------------------------------------
iteration:  35  cost_NQE:  0.414915693904020

In [7]:
def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

prediction_NQE = [QCNN_classifier(weights_NQE, x, data_type) for x in X_test]
prediction_TQE = [data_reuploading_QCNN_classifier(weights_TQE, x, data_type) for x in X_test]


accuracy_NQE = accuracy_test(prediction_NQE, Y_test)
accuracy_TQE = accuracy_test(prediction_TQE, Y_test)

print(f'NQE Accuracy: {accuracy_NQE}')
print(f'TQE Accuracy: {accuracy_TQE}')


NQE Accuracy: 0.9744680851063829
TQE Accuracy: 0.6520094562647755


In [8]:
feature_reduction = 'PCA8'
classes = [0,1]
data_type = 'fashion'
X_train, X_test, Y_train, Y_test = data.data_load_and_process(data_type, feature_reduction=feature_reduction, classes=classes)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [9]:
loss_history_NQE, loss_history_TQE, weights_NQE, weights_TQE = circuit_training(X_train, Y_train, data_type)
f = open('weights and losses fashion.txt', 'a')
f.write(f'Loss History NQE:')
f.write('\n')
f.write(str(loss_history_NQE))
f.write('\n')
f.write(f'Loss History TQE:')
f.write('\n')
f.write(str(loss_history_TQE))
f.write('\n')
f.write(f'Weights NQE:')
f.write('\n')
f.write(str(weights_NQE))
f.write('\n')
f.write(f'Weights TQE:')
f.write('\n')
f.write(str(weights_TQE))
f.write('\n')
f.close()

--------------------------------------
iteration:  0  cost_NQE:  0.5415045395590142
iteration:  0  cost_TQE:  0.5012970168612193
--------------------------------------
iteration:  5  cost_NQE:  0.5405317769130998
iteration:  5  cost_TQE:  0.5090791180285343
--------------------------------------
iteration:  10  cost_NQE:  0.5017509727626459
iteration:  10  cost_TQE:  0.5214007782101168
--------------------------------------
iteration:  15  cost_NQE:  0.4819066147859923
iteration:  15  cost_TQE:  0.49571984435797667
--------------------------------------
iteration:  20  cost_NQE:  0.438780804150454
iteration:  20  cost_TQE:  0.48975356679636833
--------------------------------------
iteration:  25  cost_NQE:  0.40434500648508437
iteration:  25  cost_TQE:  0.47756160830090794
--------------------------------------
iteration:  30  cost_NQE:  0.4088197146562905
iteration:  30  cost_TQE:  0.4880025940337224
--------------------------------------
iteration:  35  cost_NQE:  0.3846952010376135

In [10]:
prediction_NQE = [QCNN_classifier(weights_NQE, x, data_type) for x in X_test]
prediction_TQE = [data_reuploading_QCNN_classifier(weights_TQE, x, data_type) for x in X_test]


accuracy_NQE = accuracy_test(prediction_NQE, Y_test)
accuracy_TQE = accuracy_test(prediction_TQE, Y_test)

print(f'NQE Accuracy: {accuracy_NQE}')
print(f'TQE Accuracy: {accuracy_TQE}')

NQE Accuracy: 0.943
TQE Accuracy: 0.612
